In [ ]:
#https://www.kaggle.com/jerrykuo7727/word2vec#%E3%80%88Gensim-Word2Vec-%E7%B0%A1%E6%98%93%E6%95%99%E5%AD%B8%E3%80%89
#http://zake7749.github.io/2016/08/28/word2vec-with-gensim/
#https://medium.com/pyladies-taiwan/%E8%87%AA%E7%84%B6%E8%AA%9E%E8%A8%80%E8%99%95%E7%90%86%E5%85%A5%E9%96%80-word2vec%E5%B0%8F%E5%AF%A6%E4%BD%9C-f8832d9677c8
'''
Word2Vec:非監督學習，理論上資料及越大越好
Word2vec 依靠了 skip-gram 與 Continuous Bag of Word (CBOW) 的方法來實作，核心是一個極為淺層的類神經網路。
透過使每個字詞與前後字詞的向量相近，來訓練出含有每個字詞語義的字詞向量
Word2vec 對於文字處理的領域帶來了巨大的改變，
後面也可以再使用 Convolution Neural Network 來做進階的辨識等方法。

但如果將一份文件中每個字詞的向量加總，試圖來獲得這份文章的 vector 時，
依舊是會發生失去了順序性的問題，下一篇我將介紹 Doc2vec 這方法。
'''

In [1]:
import os
import jieba
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from gensim.models.word2vec import Word2Vec


In [2]:
#輸入資料集
def process_dirs(base_dir):
    df = pd.DataFrame(columns=["cat", "text"])
    
    for dir_path, dir_name, file_names in os.walk(base_dir):
        for single_file in file_names:
                try:
                    f = open(os.path.join(dir_path, single_file), "r", encoding="utf-8")
                    content = f.read()
                    content = content.replace("\r",'').replace("\n", '')
                    split_word = jieba.lcut(content)
                    s = pd.Series([dir_path.split("\\")[1],split_word], index=["cat","text"])
                    df = df.append(s, ignore_index = True)
                except UnicodeDecodeError:
                    pass
    df['cat'] = df['cat'].astype('category')
    return df



In [3]:
base_dir= 'chinese_news_trans'
test_dir = 'chinese_news_test'
train_df = process_dirs(base_dir)
test_df = process_dirs(test_dir)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\keneau\AppData\Local\Temp\jieba.cache
Loading model cost 3.446 seconds.
Prefix dict has been built succesfully.


In [4]:
corpus = pd.concat([train_df.text, test_df.text]).sample(frac=1)
len(corpus)

2738

In [5]:
#使用sample，frac=1的時候對於行做shuffle
corpus = pd.concat([train_df.text, test_df.text]).sample(frac=1)
corpus_test = corpus.head(100)


In [6]:
model = Word2Vec(corpus_test)

In [7]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [8]:
most_similar(model, ['伊拉克', '和平', '導演'],topn=10)

伊拉克 not found in Word2Vec model!
導演 not found in Word2Vec model!


,和平,cos
0,們,0.998802
1,幹,0.998707
2,專家,0.998703
3,同,0.998604
4,聯合,0.998564
5,最高,0.998486
6,第四,0.998384
7,行業,0.998332
8,組建,0.998324
9,愛,0.998245


In [ ]:
#Word2Vec 參數調整指南
#ref:https://radimrehurek.com/gensim/models/word2vec.html
'''
準備一個訓練快速的陽春模型，先把feature處理到你滿意為止再開始調整模型 
feature決定了準確度的上限，調整模型只是讓我們盡可能接近這個上限而已 
Kaggler投入在資料探索與feature engineering的時間，往往會是調整模型的兩倍以上 
'''

In [ ]:
#size=100：詞向量的維度大小，維度太小會無法有效表達詞與詞的關係，維度太大會使關係太稀疏而難以找出規則 
#Kaggle比賽上常用的詞向量維度介於200到300之間
#iter=5：訓練的回數，訓練過少會使得詞關係過為鬆散，訓練過度又會使得詞關係過為極端
#當使用較大的詞向量維度時，可能會需要訓練更多次，我們先用iter=10來看看結果
#min_count=5：出現次數大於等於min_count的詞，才會納入Word2Vec的詞典中
#window=5：CBOW下決定Word2Vec一次取多少詞來預測中間詞
#max_vocab_size=None：Word2Vec的詞典容納上限，出現次數最低的詞會優先被剔除
#hs=0：hs=0時採用Negative Sampling，hs=1時採用Hierarchical Softmax
#workers=3：訓練用的線程數量（可以加快訓練速度）

In [9]:
model = Word2Vec(corpus,iter=5,size=200,window=5,min_count=5,workers=4)

In [10]:
most_similar(model, ['伊拉克', '和平', '導演'],topn=10)

,伊拉克,cos,和平,cos,導演,cos
0,此次,0.818974,中東,0.946329,西洋,0.950848
1,戰爭,0.807849,兩國,0.907003,散文,0.944279
2,海灣,0.796979,外交,0.876159,京劇,0.940952
3,伊朗,0.774241,友誼,0.875554,古典,0.940103
4,演習,0.762786,各國,0.868877,曲藝,0.938228
5,士兵,0.760707,歐洲,0.859499,神韻,0.937542
6,布什,0.760571,民主,0.854557,唱法,0.937123
7,越南,0.748433,阿拉伯,0.853337,幼兒,0.935728
8,敘利亞,0.736598,柬埔寨,0.852852,詩歌,0.934469
9,美軍事,0.728926,共同,0.852595,佳作,0.933675


In [ ]:
#w2vref:https://medium.com/life-of-small-data-engineer/%E8%83%BD%E8%A2%AB%E9%9B%BB%E8%85%A6%E7%90%86%E8%A7%A3%E7%9A%84%E6%96%87%E5%AD%97-nlp-%E4%B8%80-word-embedding-4146267019cb
#Bag of Words (BoW):是一種將句子拆解成每個詞出現次數的方法但是卻忽略掉了很重要的次序關係
#Word Embedding:概念是建立字詞向量，將句子中每個字轉換為向量，最後結合起來變成矩陣。
'''
Word Embedding可以保留句子中字詞的順序，且改變為適合電腦運算的矩陣形式，但產生了新的問題稀疏矩陣「當字詞過多時每個字的向量會太長，轉換的矩陣亦會更加龐大，且這矩陣中有大量的欄位皆為 0」。
依照 stanfordnlp/GloVe 預先訓練好的 word vector 中，可以用 300 維的向量來表示兩百二十萬個字詞，可以有效解決上述的維度爆炸問題，節省了大量的運算及儲存成本。
這麼一來使得電腦多了許多額外的運算及儲存成本，而且整體的效率十分低，後來有了更好的方法(Word2vec)。
'''